In [1]:
'''
    20181227 o2o大赛第1版，简单入门
'''

import os, sys, pickle

import numpy as np
import pandas as pd
import math

from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from  sklearn.preprocessing import MinMaxScaler

In [2]:
#加载数据开始
dfoff = pd.read_csv('data/ccf_offline_stage1_train.csv')
dfon = pd.read_csv('data/ccf_online_stage1_train.csv')

dftest = pd.read_csv('data/ccf_offline_stage1_test_revised.csv')

#pd.set_option('display.max_columns', None)   #设置显示的最大列，参考地址：https://blog.csdn.net/yuanxiang01/article/details/79479938
#在jupyter中不需要的上面的解决方法，就可以显示全部的数据

dfoff.head(5)   #这里直接显示数据即可，显示效果还特别好看，这样话，比

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


In [3]:
#简单统计
print('有优惠卷，购买产品：%d' % dfoff[(dfoff['Date_received']!='null') & (dfoff['Date']!='null')].shape[0])
print('有优惠卷，未购产品：%d' % dfoff[(dfoff['Date_received']!='null') & (dfoff['Date']=='null')].shape[0])
print('无优惠卷，购买产品：%d' % dfoff[(dfoff['Date_received']=='null') & (dfoff['Date']!='null')].shape[0])
print('无优惠卷，未购产品：%d' % dfoff[(dfoff['Date_received']=='null') & (dfoff['Date']=='null')].shape[0])

有优惠卷，购买产品：75382
有优惠卷，未购产品：977900
无优惠卷，购买产品：701602
无优惠卷，未购产品：0


In [4]:
#特征提取
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


In [5]:
#折扣率：有3种情况：'nan或null'表示没有打折，[0,1]表示折扣率，x:y表示满x减y
print('Discount_rate类型：\n', dfoff['Discount_rate'].unique())

Discount_rate类型：
 ['null' '150:20' '20:1' '200:20' '30:5' '50:10' '10:5' '100:10' '200:30'
 '20:5' '30:10' '50:5' '150:10' '100:30' '200:50' '100:50' '300:30' '50:20'
 '0.9' '10:1' '30:1' '0.95' '100:5' '5:1' '100:20' '0.8' '50:1' '200:10'
 '300:20' '100:1' '150:30' '300:50' '20:10' '0.85' '0.6' '150:50' '0.75'
 '0.5' '200:5' '0.7' '30:20' '300:10' '0.2' '50:30' '200:100' '150:5']


In [6]:
#处理方式:4个函数
'''
    getDiscountType()：转换打折类型
    convertRate():折扣率
    getDiscountMan():满多少
    getDiscountJian():减多少
'''


#转换打折类型，有3种情况：'nan或null'表示没有打折，[0,1]表示折扣率，x:y表示满x减y
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

#折扣率
def convertRate(row):
    '''Convert discount to rate'''
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

#满多少
def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

#减多少
def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

#处理数据的函数
def processData(df):
    #转换折扣率
    df['discount_type'] = df['Discount_rate'].apply(getDiscountType)   #扩展1列
    df['discount_rate'] = df['Discount_rate'].apply(convertRate)
    df['discount_man'] = df['Discount_rate'].apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].apply(getDiscountJian)

    print(df['discount_rate'].unique())

    return df

In [7]:
dfoff = processData(dfoff)
dftest = processData(dftest)

dfoff.head(5)

[ 1.          0.86666667  0.95        0.9         0.83333333  0.8         0.5
  0.85        0.75        0.66666667  0.93333333  0.7         0.6
  0.96666667  0.98        0.99        0.975       0.33333333  0.2         0.4       ]
[ 0.83333333  0.9         0.96666667  0.8         0.95        0.75        0.98
  0.5         0.86666667  0.6         0.66666667  0.7         0.85
  0.33333333  0.94        0.93333333  0.975       0.99      ]


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,discount_jian
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0,0
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,20
2,1439408,2632,8591,20:1,0,20160217,null,1,0.950000,20,1
3,1439408,2632,1078,20:1,0,20160319,null,1,0.950000,20,1
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,1


In [8]:
#距离
print('Distance类型：', dfoff['Distance'].unique())

#将距离str转为int
dfoff['distance'] = dfoff['Distance'].replace('null', -1).astype(int)
print(dfoff['distance'].unique())   #unique()去掉重复的数据
dftest['distance'] = dfoff['Distance'].replace('null', -1).astype(int)
print(dftest['distance'].unique())   #unique()去掉重复的数据

Distance类型： ['0' '1' 'null' '2' '10' '4' '7' '9' '3' '5' '6' '8']
[ 0  1 -1  2 10  4  7  9  3  5  6  8]
[ 0  1 -1  2 10  4  7  9  3  5  6  8]


In [9]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,discount_jian,distance
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0,0,0
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,20,1
2,1439408,2632,8591,20:1,0,20160217,null,1,0.950000,20,1,0
3,1439408,2632,1078,20:1,0,20160319,null,1,0.950000,20,1,0
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,1,0


领券日期

In [10]:
date_received = dfoff['Date_received'].unique()
date_received = sorted(date_received[date_received!='null'])

date_buy = dfoff['Date'].unique()
date_buy = sorted(date_buy[date_buy!='null'])

print('优惠券收到的日期从：', date_received[0], '到', date_received[-1])
print('优惠券消费的日期从：', date_buy[0], '到', date_buy[-1])

优惠券收到的日期从： 20160101 到 20160615
优惠券消费的日期从： 20160101 到 20160630


# 关于领券日期的特征：
*    weekday:{null,1,2,3,4,5,6,7}
*    weekday_type:{1,0} {1-周六，周日领取的优惠券，0-周一到周五领取的优惠券}
*    weekday_1:{1 0 0 0 0 0 0}   #独热编码表示一天正常
*    weekday_2:{0 1 0 0 0 0 0}   #独热编码表示一天正常
*    weekday_3:{0 0 1 0 0 0 0}   #独热编码表示一天正常
*    weekday_4:{0 0 0 1 0 0 0}   #独热编码表示一天正常
*    weekday_5:{0 0 0 0 1 0 0}   #独热编码表示一天正常
*    weekday_6:{0 0 0 0 0 1 0}   #独热编码表示一天正常
*    weekday_7:{0 0 0 0 0 0 1}   #独热编码表示一天正常

In [11]:
def getWeekday(row):
    if row == 'null':
        return row
    else:
        return date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1
    
dfoff['weekday'] = dfoff['Date_received'].astype(str).apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].astype(str).apply(getWeekday)

#weekday_type: 周六，周日为1，其它为0
dfoff['weekday_type'] = dfoff['weekday'].apply(lambda x:1 if x in [6,7] else 0)
dftest['weekday_type'] = dftest['weekday'].apply(lambda x:1 if x in [6,7] else 0)

#change weekday to one-hot encoding
weekdaycols = ['weekday_'+str(i) for i in range(1, 8)]
#print(weekdaycols)

tmpdf = pd.get_dummies(dfoff['weekday'].replace('null', np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'].replace('null', np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

In [12]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,...,distance,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0,...,0,null,0,0,0,0,0,0,0,0
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,...,1,6,1,0,0,0,0,0,1,0
2,1439408,2632,8591,20:1,0,20160217,null,1,0.950000,20,...,0,3,0,0,0,1,0,0,0,0
3,1439408,2632,1078,20:1,0,20160319,null,1,0.950000,20,...,0,6,1,0,0,0,0,0,1,0
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,...,0,1,0,1,0,0,0,0,0,0


## 所有特征
* discount_rate
* discount_type
* discount_man
* discount_jian
* distance
* weekday
* weekday_type
* weekday_1
* weekday_2
* weekday_3
* weekday_4
* weekday_5
* weekday_6
* weekday_7

### 标签标注
#### 三种情况
* Date_received == 'null': 表示没有领到优惠券，无需考虑，y=-1
* (Date_received != 'null') & (Date!='null') & (Date - Date_received <= 15): 表示领取优惠券在15内使用，即正样本，y=1
* (Date_received != 'null') & (Date=='null') & (Date - Date_received > 15): 表示领取优惠券未在15内使用，即负样本，y=0

#### 定义标签备注函数：

In [13]:
def label(row):
    if row['Date_received'] == 'null':
        return -1
    if row["Date"]!='null':
        td = pd.to_datetime(row['Date'], format='%Y%m%d') - pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td<=pd.Timedelta(15, 'D'):  #差别的天数
            return 1
    return 0

dfoff['label'] = dfoff.apply(label, axis=1)

In [14]:
print(dfoff['label'].value_counts())

 0    988887
-1    701602
 1     64395
Name: label, dtype: int64


In [15]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,...,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,label
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0,...,null,0,0,0,0,0,0,0,0,-1
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,...,6,1,0,0,0,0,0,1,0,0
2,1439408,2632,8591,20:1,0,20160217,null,1,0.950000,20,...,3,0,0,0,1,0,0,0,0,0
3,1439408,2632,1078,20:1,0,20160319,null,1,0.950000,20,...,6,1,0,0,0,0,0,1,0,0
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,...,1,0,1,0,0,0,0,0,0,0


## 建立线性模型 SGDClassifier
* 使用上面提取的14个特征
* 训练集：20160101-20160515；验证集：20160516-20160615
* 用线性模型 SGDClassifier

### 划分训练集/验证集

In [16]:
# data split
df = dfoff[dfoff['label']!=-1].copy()  #去掉标签为-1的数据
train = df[(df['Date_received']<'20160516')].copy()
valid = df[(df['Date_received']>'20160516') & (df['Date_received']<='20160615')].copy()
print('Train Set: \n', train['label'].value_counts())
print('Valid Set: \n', valid['label'].value_counts())

Train Set: 
 0    759172
1     41524
Name: label, dtype: int64
Valid Set: 
 0    225182
1     22312
Name: label, dtype: int64


In [17]:
# 特征数据
original_feature = ['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'distance', 'weekday', 'weekday_type'] + weekdaycols
print('共有 特征：', len(original_feature), '个')
print(original_feature)

共有 特征： 14 个
['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


### 建立模型

In [24]:
def check_model(data, predictors):
    # 注意调用函数中的参数与值之间的=两侧不能出现空格
    classifier = lambda: SGDClassifier(
        loss='log',   #loss function: logistic regression
        penalty='elasticnet', #L1 & L2
        fit_intercept=True,   #是否存在截距，默认存在
        #max_iter=100,  #使用这个参数会有问题，注释即可以解决问题
        shuffle=True,  #Whether or not the training data should be shuffled after each epoch
        n_jobs=1, #The number of processors to use
        class_weight=None)  #Weights associated with classes. If not given, all classes are supposed to have weight 1
    
    #管道机制使得参数集在新数据集（比如测试集）上的重复使用，管道机制实现了全部步骤的流式化封装和管理
    model = Pipeline(steps=[
        ('ss', StandardScaler()),  #transformer
        ('en', classifier())    #estimator
    ])
    
    #这里会出现ValueError:Invalid parameters的异常
    #解决方案：https://blog.csdn.net/quiet_girl/article/details/72811807，不能使用一个下划线，要使用双下划线
    parameters={
        'en__alpha': [0.001, 0.01, 0.1],  #自己不知道选择哪个参数，给定一个列表,双下划线
        'en__l1_ratio': [0.001, 0.01, 0.1]  #自己不知道选择哪个参数，给定一个列表,双下划线
    }
    
    #StratifiedKFold用法类似Kfold，但是他是分层采样，确保训练集，测试集中各类样本的比例与原始数据集中相同
    folder = StratifiedKFold(n_splits=3, shuffle=True)
    
    #Exhaustive search over specified parameter values for an estimator
    grid_search = GridSearchCV(
        model,
        parameters,
        cv=folder,
        n_jobs=-1,  #-1 means using all processors
        verbose=1)
    grid_search = grid_search.fit(data[predictors], data['label'])
    
    return grid_search

### 训练

In [29]:
predictors = original_feature
model = check_model(train, predictors)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   23.0s finished


### 验证
* 对验证集中每个优惠券预测的结果计算AUC，再对所有优惠券的AUC求平均。计算AUC的时候，如果label只有一类，就直接跳过，因为AUC无法计算。

In [31]:
# valid predict
y_valid_pred = model.predict_proba(valid[predictors])
valid1 = valid.copy()
valid1['pred_prob']=y_valid_pred[:,1]
valid1.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,...,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,label,pred_prob
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,...,1,0,0,0,0,0,1,0,0,0.018746
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,...,0,1,0,0,0,0,0,0,0,0.102923
9,2029232,450,1532,30:5,0,20160530,null,1,0.833333,30,...,0,1,0,0,0,0,0,0,0,0.096979
10,2029232,6459,12737,20:1,0,20160519,null,1,0.950000,20,...,0,0,0,0,1,0,0,0,0,0.138164
13,2747744,6901,1097,50:10,null,20160606,null,1,0.800000,50,...,0,1,0,0,0,0,0,0,0,0.089292


#### 计算AUC

In [32]:
vg = valid1.groupby(['Coupon_id'])
aucs=[]
for i in vg:
    tmpdf= i[1]
    if len(tmpdf['label'].unique())!=2:
        continue
    fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred_prob'], pos_label=1)
    aucs.append(auc(fpr, tpr))
print(np.average(aucs))

0.535746617064


### 测试

In [33]:
y_test_pred = model.predict_proba(dftest[predictors])
dftest1 = dftest[['User_id', 'Coupon_id', 'Date_received']].copy()
dftest1['Probability']=y_test_pred[:,1]
dftest1.to_csv('submit1.csv', index=False, header=False)
dftest1.head(5)

,User_id,Coupon_id,Date_received,Probability
0,4129537,9983,20160712,0.132630
1,6949378,3429,20160706,0.108338
2,2166529,6928,20160727,0.017622
3,2166529,1808,20160727,0.058612
4,6172162,6500,20160708,0.099093


#### 保存模型 & 导入模型

In [34]:
if not os.path.isfile('1_model.pkl'):
    with open('1_model.pkl', 'wb') as f:
        pickle.dump(model, f)
else:
    with open('1_model.pkl', 'rb') as f:
        model = pickle.load(f)

### 优化模型......
* 特征工程
* 机器学习算法
* 模型集成